In [1]:
import sys
import os
import glob
import struct
import binascii

import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import pandas as pd

# import dateutil
# from scipy import __version__ as scipy_version
# import scipy.interpolate
# import scipy.stats
# from statsmodels import __version__ as statsmodels_version
# import statsmodels.formula.api as smformula

from collections import OrderedDict
from IPython.display import display, HTML

print(sys.version)
print('numpy', np.__version__)
print('matplotlib', mpl.__version__)
print('pandas',pd.__version__)
# print('scipy', scipy_version)
# print('statsmodels', statsmodels_version)

3.6.3 |Anaconda custom (64-bit)| (default, Oct 15 2017, 03:27:45) [MSC v.1900 64 bit (AMD64)]
numpy 1.13.3
matplotlib 2.2.2
pandas 0.20.3


In [2]:
# Setup plot style
%matplotlib notebook
plt.style.use('seaborn-paper')
mpl.rcParams['figure.facecolor'] = (0.8, 0.8, 0.8, 1)

In [3]:
# Misc utility functions
def tempF2C(x): return (x-32.0)*5.0/9.0
def tempC2F(x): return (x*9.0/5.0)+32.0


In [4]:
# Parameters
# fn = r'C:\Users\Travis Collier\Desktop\proj\ddtool\testfiles\USDA-WS6\20148981___Over_the_last_week_2018_04_30_22_07_39_UTC_1_pt1.csv'
TEMPERATURE_FILENAME = r'../testfiles/single_file.csv'

HOUR_OFFSET = -7  # UTC to localtime offset so days are started at local midnight (not worrying about DST)
BASE_TEMP = tempF2C(54.3)
DD_PER_GEN = 622*5/9 # generation time in cumulative degree days

In [5]:
df = pd.read_csv(TEMPERATURE_FILENAME, parse_dates=['Date']).dropna()
tcol = [x for x in df.columns if x.startswith('Temperature ')]
if len(tcol) < 1:
    print("ERROR: Temperature column not found", file=sys.stderr)
else:
    tmp = [x.strip() for x in tcol[0].split(',')]
    station = tmp[-1]
    print(tmp, station)
    
    t = df.loc[:,['Date',tcol[0]]]
    t.set_index('Date', inplace=True)
    t.columns = ['T']
    t.sort_index(inplace=True)
    #t['station'] = station
    first = t.index[0]
    last = t.index[-1]
    
    print(station, first, last)
display(t.shape)
display(t.head())
df = t.copy(deep=True)

# @TCC TEMP : test handling of missing data
# df.drop(df.loc['2018-04-25 07:00:00':'2018-04-27 06:59:59'].index, inplace=True)

['Temperature (S-THB 20148981:20147576-1)', '*C', 'USDA-WS6'] USDA-WS6
USDA-WS6 2018-04-24 07:00:00 2018-04-30 06:55:00


(1728, 1)

,T
Date,
2018-04-24 07:00:00,20.32
2018-04-24 07:05:00,20.34
2018-04-24 07:10:00,20.34
2018-04-24 07:15:00,20.34
2018-04-24 07:20:00,20.36


In [6]:
t = df.copy(deep=True)
t.index = t.index+pd.Timedelta(HOUR_OFFSET, unit='h')
display(t.head())

grp = t.groupby(pd.Grouper(freq='D'))
tmin = np.min(grp)
tmax = np.max(grp)
mmt = tmin.merge(tmax, left_index=True, right_index=True, suffixes=['min','max'])
mmt['Tave'] = (tmin+tmax)/2.0
mmt['N'] = grp.count()
mmt

,T
Date,
2018-04-24 00:00:00,20.32
2018-04-24 00:05:00,20.34
2018-04-24 00:10:00,20.34
2018-04-24 00:15:00,20.34
2018-04-24 00:20:00,20.36


,Tmin,Tmax,Tave,N
Date,,,,
2018-04-24,18.08,25.77,21.925,288
2018-04-25,18.06,26.52,22.290,288
2018-04-26,18.46,23.06,20.760,288
2018-04-27,18.75,26.72,22.735,288
2018-04-28,19.10,24.92,22.010,288
2018-04-29,18.70,21.41,20.055,288


In [7]:
fig = plt.figure()
ax = fig.add_subplot(1,1,1)

x = np.dstack((mmt.index.values, (mmt.index+pd.Timedelta(1,unit='d')).values)).flatten()
tmin = mmt['Tmin'].values.repeat(2)
tmax = mmt['Tmax'].values.repeat(2)
ax.fill_between(x, tmin, tmax, color='gray', alpha=0.5)
# ax.plot(x, tmin, 'b.-')#color='blue', alpha=0.5)
# ax.plot(x, tmax, 'r.-')#color='red', alpha=0.5)

ax.plot(t.index, t['T'])

<IPython.core.display.Javascript object>

TODO : figure out a way to drop partial days...

In [8]:
# Function which computes BM (single sine method) degree day generation from temperature data
def compute_BMDD_Fs(tmin, tmax, base_temp, dd_gen=None, skipna=False):
    def _compute_daily_BM_DD(mint, maxt, avet, base_temp):
        """Use standard Baskerville-Ermin (single sine) degree-day method
        to compute the degree-day values for each a single day.
        """
        if avet is None:
            avet = (mint+maxt)/2.0 # simple midpoint (like in the refs)
        dd = np.nan # value which we're computing
        # Step 1: Adjust for observation time; not relevant
        # Step 2: GDD = 0 if max < base (curve all below base)
        if maxt < base_temp:
            dd = 0
        # Step 3: Calc mean temp for day; already done previously
        # Step 4: min > base; then whole curve counts
        elif mint >= base_temp:
            dd = avet - base_temp
        # Step 5: else use curve minus part below base
        else:
            W = (maxt-mint)/2.0
            tmp = (base_temp-avet) / W
            if tmp < -1:
                print('WARNING: (base_temp-avet)/W = {} : should be [-1:1]'.format(tmp))
                tmp = -1
            if tmp > 1:
                print('WARNING: (base_temp-avet)/W = {} : should be [-1:1]'.format(tmp))
                tmp = 1
            A = np.arcsin(tmp)
            dd = ((W*np.cos(A))-((base_temp-avet)*((np.pi/2.0)-A)))/np.pi
        return dd
    # compute the degree-days for each day in the temperature input (from tmin and tmax vectors)
    dd = pd.concat([tmin,tmax], axis=1)
    dd.columns = ['tmin', 'tmax']
    dd['DD'] = dd.apply(lambda x: _compute_daily_BM_DD(x[0], x[1], (x[0]+x[1])/2.0, base_temp), axis=1)
    dd['DD_cum'] = dd['DD'].cumsum(skipna=skipna)   
    # add generation number if we are given DD per generation (dd_gen)
    if dd_gen:
        dd['gen'] = np.floor(dd['DD_cum']/dd_gen).astype(int)
    return dd

In [9]:
dd = compute_BMDD_Fs(mmt['Tmin'], mmt['Tmax'], BASE_TEMP, 20)#DD_PER_GEN)
dd

,tmin,tmax,DD,DD_cum,gen
Date,,,,,
2018-04-24,18.08,25.77,9.536111,9.536111,0
2018-04-25,18.06,26.52,9.901111,19.437222,0
2018-04-26,18.46,23.06,8.371111,27.808333,1
2018-04-27,18.75,26.72,10.346111,38.154444,1
2018-04-28,19.10,24.92,9.621111,47.775556,2
2018-04-29,18.70,21.41,7.666111,55.441667,2


In [10]:
np.floor(dd['DD_cum']/20).astype(int)

Date
2018-04-24    0
2018-04-25    0
2018-04-26    1
2018-04-27    1
2018-04-28    2
2018-04-29    2
Freq: D, Name: DD_cum, dtype: int32

# Exploring CDFA binary temperature files

In [11]:
FN = '../../CDFA_old_dd/LAAR18'



In [12]:
def msbin2ieee(msbin):
    """
    Convert an array of 4 bytes containing Microsoft Binary floating point
    number to IEEE floating point format (which is used by Python)
    adapted from: https://github.com/choonkeat/ms2txt/blob/master/metastock/utils.py
    """
    as_int = struct.unpack("i", msbin)
    if not as_int:
        return 0.0
    man = int(struct.unpack('H', msbin[2:])[0])
    if not man:
        return 0.0
    exp = (man & 0xff00) - 0x0200
    man = man & 0x7f | (man << 8) & 0x8000
    man |= exp >> 1
    ieee = msbin[:2]
    ieee += bytes([man & 0xFF])
    ieee += bytes([(man >> 8) & 0xFF])
#     print(ieee)
    return struct.unpack("f", ieee)[0]

display(msbin2ieee(b'\xDB\x0F\x49\x81')*2) # should ~= pi

display(msbin2ieee(b'\xCD\xCC\x2A\x87'))


3.1415927410125732

85.4000015258789

In [13]:
# 3 AS N$, 2 AS D$, 2 AS M$, 2 AS Y$, 5 AS M2$, 5 AS M3$, 16 AS X$
date_fmt = r"=hchhh" # 5 5 16"
date_size = struct.calcsize(date_fmt)

with open(FN,'rb') as fh:
    while True:
        date_bytes = fh.read(date_size)
        if len(date_bytes) < date_size:
            if len(date_bytes) > 0:
                print("WARNING: Some data left in file")
            break
        (n, _, d, m, y) = struct.unpack(date_fmt, date_bytes)
        tmin = msbin2ieee(fh.read(4))
        assert fh.read(1) == b' ' # This byte should be a space (0x20)
        tmax = msbin2ieee(fh.read(4))
        assert fh.read(1) == b' ' # This byte should be a space (0x20)
        fooX = fh.read(16) # I have no clue what this value is
        print(n, d, m, y, tmin, tmax, fooX, sep='\t')
        _ = fh.read(128-35) # skip to next record in the file (I have no idea why each record is 128 bytes)
    

1	1	1	18	48.5	71.9000015258789	b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00'
2	2	1	18	52.79999923706055	72.19999694824219	b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00'
3	3	1	18	57.099998474121094	71.5999984741211	b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00'
4	4	1	18	54.20000076293945	75.0	b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00'
5	5	1	18	53.900001525878906	71.80000305175781	b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00'
6	6	1	18	57.599998474121094	68.30000305175781	b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00'
7	7	1	18	56.70000076293945	72.5	b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00'
8	8	1	18	60.5	64.4000015258789	b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00'
9	9	1	18	51.20000076293945	60.5	b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00'
10	10	1	18	47.79999923706055	66.30000305175781	b